In [23]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.set_option('plotting.backend', 'plotly')
from datetime import date, datetime as dt
import os, re
import plotly.colors
import plotly.graph_objects as go, plotly.express as px, plotly.figure_factory as ff
from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots

from statsmodels.stats.proportion import proportion_confint

from tennis_utils.player import TennisDataLoader, TennisPlayerDataLoader, TennisPlayer, TennisPlayerRenderer
from tennis_utils.scrapers import SackmanScraper


def get_player_name(name):

    return '. '.join(['.'.join([e[0] for e in name.split(' ')[:-1]]), name.split(' ')[-1]])


ModuleNotFoundError: No module named 'tennis_utils.scrapers'

In [ ]:

player_name = 'Roger Federer'
data_path = os.getcwd()+'/data'

tdl = TennisDataLoader(data_path+'/matches.parquet', data_path+'/players.parquet')
matches_df, players_df = tdl.matches, tdl.players

tpdl = TennisPlayerDataLoader(player_name, matches_df, players_df)

# Subset selected player matches data
player_matches = tpdl.player_matches #matches_df[matches_df['player_name'] == player_name]
player_details = tpdl.player_details #players_df[players_df['player_name']==player_name]
player_rank = tpdl.player_rank

tp = TennisPlayerRenderer(player_name=player_name, 
                          player_matches=player_matches, 
                          player_rank=player_rank, 
                          player_details=player_details)


In [38]:
import yaml# import load, Loader, dump, Dumper

with open(os.getcwd() + '/config.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    data = yaml.load(file, Loader=yaml.Loader)

details_mapping = data['details_mapping']

In [3]:
"""
https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval
https://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportion_confint.html
https://www.ucl.ac.uk/english-usage/staff/sean/resources/binomialpoisson.pdf
"""

'\nhttps://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval\nhttps://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportion_confint.html\nhttps://www.ucl.ac.uk/english-usage/staff/sean/resources/binomialpoisson.pdf\n'